In [34]:
%pip install -U ultralytics==8.4.0 torch==2.9.1 tensorflow==2.20.0 sng4onnx==1.0.4

Note: you may need to restart the kernel to use updated packages.


In [35]:
import os

import certifi
import torch

# Ensure HTTPS requests (model download, dataset fetch, etc.) use certifi's CA bundle.
os.environ["SSL_CERT_FILE"] = certifi.where()
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()

# Pick the best available device: Apple MPS, CUDA GPU, or CPU fallback.
device = "mps" if torch.backends.mps.is_available() else (0 if torch.cuda.is_available() else "cpu")
print("device:", device)


device: mps


In [36]:
# Import Ultralytics YOLO API.
from ultralytics import YOLO


In [6]:
# Load a pretrained YOLOv11 nano checkpoint and train on the local dataset.
model = YOLO("yolo11n.pt")
model.train(
    data="../dataset_1/data.yaml",
    imgsz=640,
    epochs=1,
    batch=16,
    device=device,
)


Ultralytics 8.4.0 🚀 Python-3.12.10 torch-2.9.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../dataset_1/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x4c7412930>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    

In [37]:
# Export the trained model to a mobile-friendly TFLite format with NMS enabled.
m = YOLO("yolo11n.pt")
m.export(format="tflite", imgsz=640, half=True, nms=True)


Ultralytics 8.4.0 🚀 Python-3.12.10 torch-2.9.1 CPU (Apple M4 Pro)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 300, 6) (5.4 MB)
requirements: Ultralytics requirement ['tf_keras<=2.19.0'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

requirements: AutoUpdate success ✅ 0.5s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


TensorFlow SavedModel: starting export with tensorflow 2.20.0...

ONNX: starting export with onnx 1.20.1 opset 22...
ONNX: slimming with onnxslim 0.1.82...
ONNX: export success ✅ 0.8s, saved as 'yolo11n.onnx' (10.4 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.8...
Saved artifact at 'yolo11n_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec

I0000 00:00:1768399242.888213 4720357 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1768399242.888293 4720357 single_machine.cc:376] Starting new session
W0000 00:00:1768399243.492775 4720357 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1768399243.492789 4720357 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
I0000 00:00:1768399243.839829 4720357 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1768399243.839878 4720357 single_machine.cc:376] Starting new session
W0000 00:00:1768399244.416809 4720357 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1768399244.416822 4720357 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.


TensorFlow SavedModel: export success ✅ 8.8s, saved as 'yolo11n_saved_model' (26.1 MB)

TensorFlow Lite: starting export with tensorflow 2.20.0...
TensorFlow Lite: export success ✅ 0.0s, saved as 'yolo11n_saved_model/yolo11n_float16.tflite' (5.2 MB)

Export complete (8.9s)
Results saved to /Users/uri/LinkedInProjects/plate-detector-android/experiments/ultralytics
Predict:         yolo predict task=detect model=yolo11n_saved_model/yolo11n_float16.tflite imgsz=640 half 
Validate:        yolo val task=detect model=yolo11n_saved_model/yolo11n_float16.tflite imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml half 
Visualize:       https://netron.app


'yolo11n_saved_model/yolo11n_float16.tflite'